# import

In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import PowerTransformer

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import mlflow

THRESHOLD=50

# load

In [21]:
spotify = pd.read_csv('data/spotify_data.csv')

spotify['is_popular'] = spotify['popularity'] >= THRESHOLD
spotify = spotify[spotify['year'] != 2023]

print(spotify.columns)
spotify.describe()
# spotify.head(10)
# print(spotify)

Index(['Unnamed: 0', 'artist_name', 'track_name', 'track_id', 'popularity',
       'year', 'genre', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature', 'is_popular'],
      dtype='object')


,Unnamed: 0,popularity,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,1.121115e+06,1.121115e+06,1.121115e+06,1.121115e+06,1.121115e+06,1.121115e+06,1.121115e+06,1.121115e+06,1.121115e+06,1.121115e+06,1.121115e+06,1.121115e+06,1.121115e+06,1.121115e+06,1.121115e+06,1.121115e+06
mean,6.609827e+05,1.833009e+01,2.011574e+03,5.373502e-01,6.404769e-01,5.288010e+00,-8.958909e+00,6.350294e-01,9.296653e-02,3.207728e-01,2.512347e-01,2.232447e-01,4.566433e-01,1.213677e+02,2.503135e+05,3.885708e+00
std,4.357414e+05,1.580787e+01,6.598368e+00,1.844130e-01,2.698937e-01,3.554551e+00,5.646161e+00,4.814221e-01,1.275915e-01,3.545309e-01,3.644375e-01,2.015088e-01,2.685501e-01,2.977975e+01,1.501637e+05,4.678041e-01
min,0.000000e+00,0.000000e+00,2.000000e+03,0.000000e+00,0.000000e+00,0.000000e+00,-5.810000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.073000e+03,0.000000e+00
25%,2.802785e+05,5.000000e+00,2.006000e+03,4.130000e-01,4.550000e-01,2.000000e+00,-1.081400e+01,0.000000e+00,3.710000e-02,6.340000e-03,1.030000e-06,9.780000e-02,2.270000e-01,9.879100e+01,1.818670e+05,4.000000e+00
50%,5.605570e+05,1.500000e+01,2.012000e+03,5.500000e-01,6.940000e-01,5.000000e+00,-7.441000e+00,1.000000e+00,5.070000e-02,1.460000e-01,1.710000e-03,1.340000e-01,4.390000e-01,1.218790e+02,2.263870e+05,4.000000e+00
75%,1.049246e+06,2.900000e+01,2.017000e+03,6.770000e-01,8.730000e-01,8.000000e+00,-5.272000e+00,1.000000e+00,8.900000e-02,6.380000e-01,6.080000e-01,2.920000e-01,6.750000e-01,1.398940e+02,2.876400e+05,4.000000e+00
max,1.473395e+06,9.400000e+01,2.022000e+03,9.930000e-01,1.000000e+00,1.100000e+01,6.172000e+00,1.000000e+00,9.710000e-01,9.960000e-01,1.000000e+00,1.000000e+00,1.000000e+00,2.499930e+02,6.000495e+06,5.000000e+00


# train model

In [27]:
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment("Spotify popular song prediction")
mlflow.autolog()

numeric = spotify['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
categorical = spotify['artist_name', 'track_name', 'year', 'genre', 'key', 'time_signature']

X = numeric + categorical
y = spotify['is_popular'].copy()

model = RandomForestRegressor()

with mlflow.start_run():
    pipeline = makePipeline(model)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    pipeline.fit(X_train, y_train)

2025/05/12 16:22:02 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/05/12 16:22:02 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


KeyError: ('danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms')

# pipeline

In [26]:
def makePipeline(model):

    num_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scale', StandardScaler()),
    ])

    cat_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('scale', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
    ])

    col_transformer = ColumnTransformer(transformers=[
        ('num_pipe', num_pipe, numeric),
        ('cat_pipe', cat_pipe, categorical),
    ],
        remainder='drop',
        n_jobs=-1,
    )

    return Pipeline([
        ('col_transformer', col_transformer),
        ('model', model),
    ])

# evaluate model

In [ ]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))